# Project 3: Colab Training Runner (Reorganized)
**Pre-requisites:**
1. Switch Runtime to GPU (Runtime -> Change runtime type -> T4 GPU).
2. Upload `datasets.zip` to Google Drive folder `Project3`.
3. Have your code on GitHub.

In [ ]:
import torch
if torch.cuda.is_available():
    print(f"✅ GPU is ready: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU NOT detected. Go to Runtime -> Change runtime type -> T4 GPU")

In [ ]:
from google.colab import drive
import os
import shutil

drive.mount('/content/drive')
DRIVE_ROOT = "/content/drive/MyDrive/Project3"
os.makedirs(DRIVE_ROOT, exist_ok=True)

In [ ]:
# REPLACE THIS with your specific Repo URL
REPO_URL = "https://github.com/GurYon19/project3.git" 
REPO_NAME = "project3"

if not os.path.exists(REPO_NAME):
    print(f"Cloning {REPO_URL}...")
    !git clone {REPO_URL}
else:
    print("Repo already cloned. Pulling latest changes...")
    %cd {REPO_NAME}
    !git pull
    %cd ..

In [ ]:
import zipfile
ZIP_PATH = os.path.join(DRIVE_ROOT, "datasets.zip")
DEST_DIR = f"/content/{REPO_NAME}"

if os.path.exists(ZIP_PATH):
    print(f"📦 Unzipping data to {DEST_DIR}...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(DEST_DIR)
    print("✅ Extraction complete!")
else:
    print(f"❌ datasets.zip not found in {DRIVE_ROOT}")

## 🔍 AUTO-DETECT DATA PATH

In [ ]:
search_root = f"/content/{REPO_NAME}"
found_path = None

for root, dirs, files in os.walk(search_root):
    if "_annotations.coco.json" in files and "train" in os.path.basename(root):
        parent = os.path.dirname(root)
        found_path = parent
        print(f"✅ Found dataset at: {found_path}")
        break

if not found_path:
    print("❌ Could not automatically find dataset path.")
    found_path = "datasets/part2"
else:
    try:
        found_path = os.path.relpath(found_path, start=f"/content/{REPO_NAME}")
    except:
        pass

## 🧹 FILTER DATASET

In [ ]:
%cd /content/{REPO_NAME}
!python tools/filter_single_object.py --data-dir "{found_path}" --auto

## ♻️ RESTORE CHECKPOINT (Crucial for Resuming)

In [ ]:
# Copy checkpoints FROM Drive TO Colab workspace
drive_ckpt_dir = os.path.join(DRIVE_ROOT, "checkpoints/part2")
local_ckpt_dir = "checkpoints/part2"

if os.path.exists(drive_ckpt_dir):
    print(f"📥 Restoring checkpoints from Drive...")
    shutil.copytree(drive_ckpt_dir, local_ckpt_dir, dirs_exist_ok=True)
    print(f"✅ Restored {len(os.listdir(local_ckpt_dir))} files.")
else:
    print("⚠️ No checkpoints found on Drive to restore. Starting fresh.")

In [ ]:
# Run training
%cd /content/{REPO_NAME}

# SET THIS TO "best_model.pth" TO RESUME
RESUME_CHECKPOINT = "best_model.pth" 
# RESUME_CHECKPOINT = None 

os.environ['PYTHONPATH'] = f"/content/{REPO_NAME}" # Ensure root imports work
print(f"Starting training using data dir: {found_path}")

cmd = f"python part2/train.py --data-dir \"{found_path}\""

if RESUME_CHECKPOINT:
    cmd += f" --resume {RESUME_CHECKPOINT}"

print(f"Executing: {cmd}")
!{cmd}

In [ ]:
# --------------------------------------------------------
# BACKUP CONTINUOUSLY
# --------------------------------------------------------
dest_ckpt = os.path.join(DRIVE_ROOT, "checkpoints/part2")
os.makedirs(dest_ckpt, exist_ok=True)
if os.path.exists("checkpoints/part2"):
    shutil.copytree("checkpoints/part2", dest_ckpt, dirs_exist_ok=True)
    print(f"✅ Saved CHECKPOINTS to Drive.")

dest_logs = os.path.join(DRIVE_ROOT, "logs/part2")
os.makedirs(dest_logs, exist_ok=True)
if os.path.exists("logs/part2"):
    shutil.copytree("logs/part2", dest_logs, dirs_exist_ok=True)
    print(f"✅ Saved LOGS to Drive.")